# Utils

In [1]:
from math import sin, pi, e, log
from typing import Callable, Tuple

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from tqdm import tqdm

from itertools import product

from functools import partialmethod

tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)

In [2]:
Function1 = Callable[[float], float]
Function2 = Callable[[float, float], float]

In [3]:
def solve_linear_system(matrix: np.ndarray, right_side: np.ndarray) -> np.ndarray:
    alphas = []
    betas = []

    # Прямой ход: находим коэффициенты
    for i in range(matrix.shape[0]):
        y = matrix[i, i] if i == 0 else matrix[i, i] + matrix[i, i - 1] * alphas[i - 1]
        alphas.append(0 if i == (matrix.shape[0] - 1) else -1 * matrix[i, i + 1] / y)
        betas.append(right_side[i] / y if i == 0 else (right_side[i] - matrix[i, i - 1] * betas[i - 1]) / y)

    # Обратный ход: находим x
    reversed_solution = []
    for i in reversed(range(matrix.shape[0])):
        reversed_solution.append(
            betas[i]
            if i == (matrix.shape[0] - 1) else
            alphas[i] * reversed_solution[matrix.shape[0] - i - 2] + betas[i]
        )
    return np.array(reversed_solution[::-1])

In [4]:
def solve_heat_equation(
        kappa: float,
        f: Function2,
        line_limit: float,
        time_limit: float,
        number_of_dimensional_layers: int,
        number_of_time_layers: int,
        init_condition: Function1,
        left_condition: Function1,
        right_condition: Function1,
        sigma: float,
) -> np.ndarray:
    time_grid, tau = np.linspace(0, time_limit, number_of_time_layers, retstep=True)
    dimensional_grid, h = np.linspace(0, line_limit, number_of_dimensional_layers, retstep=True)

    heat_matrix = [
        [init_condition(point) for point in dimensional_grid],
    ]

    for i, time in tqdm(enumerate(time_grid[1:]), desc=f'{number_of_dimensional_layers} -- {number_of_time_layers}'):
        matrix = np.zeros((number_of_dimensional_layers, number_of_dimensional_layers))
        right_side = np.empty(number_of_dimensional_layers)

        matrix[0, 0] = 1
        right_side[0] = left_condition(time)

        for j, (left, middle, right) in enumerate(zip(heat_matrix[i], heat_matrix[i][1:], heat_matrix[i][2:]), start=1):
            matrix[j, j - 1] = -1 * tau * kappa * sigma
            matrix[j, j] = h ** 2 + 2 * tau * kappa * sigma
            matrix[j, j + 1] = -1 * tau * kappa * sigma

            right_side[j] = (
                    h ** 2 * middle + (1 - sigma) * (left - 2 * middle + right) * kappa * tau
                    + tau * h ** 2 * f(dimensional_grid[j], (time - tau) + sigma * tau)
            )

        matrix[number_of_dimensional_layers - 1, number_of_dimensional_layers - 1] = 1
        right_side[number_of_dimensional_layers - 1] = right_condition(time)

        heat_matrix.append(solve_linear_system(matrix, right_side))

    return np.array(heat_matrix)

In [5]:
def get_grid_by_solution(
        solution: Function2,
        line_limit: float,
        time_limit: float,
        number_of_dimensional_layers: int,
        number_of_time_layers: int,
) -> np.ndarray:
    return np.array([
        [true_solution(x, time) for x in np.linspace(0, line_limit, number_of_dimensional_layers)]
        for time in np.linspace(0, time_limit, number_of_time_layers)
    ])

In [6]:
def _get_error(true_grid: np.ndarray, actual_grid: np.ndarray) -> Tuple[np.ndarray, Tuple[int, int]]:
    error_grid = abs(true_grid - actual_grid)

    return error_grid.max(), np.unravel_index(error_grid.argmax(), actual_grid.shape)


def banchmark(
        kappa: float,
        f: Function2,
        line_limit: float,
        time_limit: float,
        init_condition: Function1,
        left_condition: Function1,
        right_condition: Function1,
        true_solution: Function2,
        number_of_layers: int,
        sigma: float,
) -> float:
    actual_grid = solve_heat_equation(
        kappa=kappa,
        f=f,
        line_limit=line_limit,
        time_limit=time_limit,
        number_of_dimensional_layers=number_of_layers,
        number_of_time_layers=number_of_layers,
        init_condition=init_condition,
        left_condition=left_condition,
        right_condition=right_condition,
        sigma=sigma
    )

    true_grid = get_grid_by_solution(true_solution, line_limit, time_limit, number_of_layers, number_of_layers)

    return _get_error(true_grid, actual_grid)[0]

In [7]:
def compare_error(data: pd.DataFrame) -> None:
    fig = px.line(data, x='n', y='error', color='sigma')

    fig.update_xaxes(title='Размерность сетки')
    fig.update_yaxes(title='Ошибка', tickformat='.2e')
    fig.update_layout(title='Зависимость ошибки от размера сетки')

    fig.show()

In [8]:
def show_heat_map(grid: np.ndarray) -> None:
    fig = px.imshow(grid.T)

    fig.update_xaxes(title='t')
    fig.update_yaxes(title='x')
    fig.update_layout(title='Карта распространения тепла')

    fig.show()

In [9]:
def compare_grids(
        actual_grid: np.ndarray,
        true_grid: np.ndarray,
        line_limit: float,
        time_limit: float,
) -> None:
    x = np.linspace(0, time_limit, actual_grid.shape[0])
    y = np.linspace(0, line_limit, actual_grid.shape[1])

    fig = go.Figure()

    fig.add_surface(x=y, y=x, z=true_grid)

    yv, xv = np.meshgrid(y, x)
    fig.add_scatter3d(y=xv.flatten(), x=yv.flatten(), z=actual_grid.flatten(), marker=dict(size=2, color="blue"), mode='markers')

    fig.update_layout(
        title='Сравнение истинного решения и найденной сетки',
        scene_zaxis_title='Температура',
        scene_yaxis_title='Время',
        scene_xaxis_title='Координата стержня',
        height=800,
    )

    fig.show()

In [10]:
def check_steps(        
        line_limit: float,
        time_limit: float,
        number_of_dimensional_layers: int,
        number_of_time_layers: int,
        kappa: float,
) -> None:
    h = line_limit / (number_of_dimensional_layers - 1)
    tau = time_limit / (number_of_time_layers - 1)

    print(f'h: {h}')
    print(f'tau: {tau}\n')

    print(f'2 * kappa * tau: {2 * kappa * tau}')
    print(f'h^2: {h ** 2}\n')

    if 2 * kappa * tau <= h ** 2:
        print(f'Условие устойчивости выполняется!\n')
    else:
        print(f'Условие устойчивости не выполняется!\n')

In [11]:
def show_error(true_grid: np.ndarray, actual_grid: np.ndarray,line_limit: float, time_limit: float) -> None:
    error, index = _get_error(true_grid, actual_grid)

    x = np.linspace(0, line_limit, actual_grid.shape[1])
    y = np.linspace(0, time_limit, actual_grid.shape[0])

    print(f'Error: {error}')
    print(f'Bad point: ({index[1]}, {index[0]}) = ({x[index[1]]}, {y[index[0]]})\n')

# First

$$u_t = u_{xx} + 2t + sin(x)$$
$$u(x, 0) = sin(x)$$
$$u(0, t) = t^2,\ u\left(\frac{3\pi}{2}, t\right) = t^2 - 1$$
$$x \in \left[0, \frac{3\pi}{2}\right],\ t \in [0, 5]$$
$$u_{\text{true}} = sin(x) + t^2$$

In [12]:
kappa = 1
f = lambda x, t: 2 * t + sin(x)

init_condition = lambda x: sin(x)
left_condition = lambda t: t ** 2
right_condition = lambda t: t ** 2 - 1

line_limit = 3 * pi / 2
time_limit = 5

true_solution = lambda x, t: sin(x) + t ** 2

In [13]:
# first_data = pd.DataFrame(product(range(50, 1000 + 1, 50), [1, 1 / 2]), columns=['n', 'sigma'])
# first_data['error'] = first_data.apply(
#     func=lambda row: banchmark(
#         kappa,
#         f,
#         line_limit,
#         time_limit,
#         init_condition,
#         left_condition,
#         right_condition,
#         true_solution,
#         int(row.n),
#         row.sigma,
#     ),
#     axis=1,
# )
# first_data.to_csv('first.csv', index=False)

first_data = pd.read_csv('first.csv')
first_data

,n,sigma,error
0,50,1.0,0.250704
1,50,0.5,0.000957
2,100,1.0,0.124315
3,100,0.5,0.000234
4,150,1.0,0.082645
5,150,0.5,0.000103
6,200,1.0,0.061896
7,200,0.5,0.000058
8,250,1.0,0.049475
9,250,0.5,0.000037


In [14]:
compare_error(first_data)

Unsupported

In [15]:
number_of_dimensional_layers = 50
number_of_time_layers = 50
sigma = 1

true_grid = get_grid_by_solution(true_solution, line_limit, time_limit, number_of_dimensional_layers, number_of_time_layers)

actual_grid = solve_heat_equation(
    kappa=kappa, 
    f=f, 
    line_limit=line_limit, 
    time_limit=time_limit, 
    number_of_dimensional_layers=number_of_dimensional_layers, 
    number_of_time_layers=number_of_time_layers, 
    init_condition=init_condition, 
    left_condition=left_condition, 
    right_condition=right_condition, 
    sigma=sigma,
)

show_error(true_grid, actual_grid, line_limit, time_limit)
show_heat_map(actual_grid)
compare_grids(actual_grid, true_grid, line_limit, time_limit)

Error: 0.2507035236028905
Bad point: (24, 49) = (2.3081088883516845, 5.0)



Unsupported

In [16]:
number_of_dimensional_layers = 30
number_of_time_layers = 380
sigma = 0

true_grid = get_grid_by_solution(true_solution, line_limit, time_limit, number_of_dimensional_layers, number_of_time_layers)

actual_grid = solve_heat_equation(
    kappa=kappa, 
    f=f, 
    line_limit=line_limit, 
    time_limit=time_limit, 
    number_of_dimensional_layers=number_of_dimensional_layers, 
    number_of_time_layers=number_of_time_layers, 
    init_condition=init_condition, 
    left_condition=left_condition, 
    right_condition=right_condition, 
    sigma=sigma,
)

show_error(true_grid, actual_grid, line_limit, time_limit)
check_steps(line_limit, time_limit, number_of_dimensional_layers, number_of_time_layers, kappa)
show_heat_map(actual_grid)
compare_grids(actual_grid, true_grid, line_limit, time_limit)

Error: 0.030210924375438708
Bad point: (15, 379) = (2.4374425760610468, 5.0)

h: 0.1624961717374031
tau: 0.013192612137203167

2 * kappa * tau: 0.026385224274406333
h^2: 0.026405005829311604

Условие устойчивости выполняется!



Unsupported

In [17]:
number_of_dimensional_layers = 30
number_of_time_layers = 369  # -11
sigma = 0

true_grid = get_grid_by_solution(true_solution, line_limit, time_limit, number_of_dimensional_layers, number_of_time_layers)

actual_grid = solve_heat_equation(
    kappa=kappa,
    f=f,
    line_limit=line_limit,
    time_limit=time_limit,
    number_of_dimensional_layers=number_of_dimensional_layers,
    number_of_time_layers=number_of_time_layers,
    init_condition=init_condition,
    left_condition=left_condition,
    right_condition=right_condition,
    sigma=sigma,
)

show_error(true_grid, actual_grid, line_limit, time_limit)
check_steps(line_limit, time_limit, number_of_dimensional_layers, number_of_time_layers, kappa)
show_heat_map(actual_grid)
compare_grids(actual_grid, true_grid, line_limit, time_limit)

Error: 3.767078225657496
Bad point: (15, 368) = (2.4374425760610468, 5.0)

h: 0.1624961717374031
tau: 0.01358695652173913

2 * kappa * tau: 0.02717391304347826
h^2: 0.026405005829311604

Условие устойчивости не выполняется!



Unsupported

# Second

$$u_t = 2u_{xx} + \left( \frac{1}{t + 1} - 2e^x \right)$$
$$u(x, 0) = e^x$$
$$u(0, t) = 1 + ln(t + 1),\ u(ln(3), t) = 3 + ln(t + 1)$$
$$x \in [0, ln(3)],\ t \in [0, 1]$$
$$u_{true} = e^x + ln(t + 1)$$

In [18]:
kappa = 2
f = lambda x, t: 1 / (t + 1) - 2 * e ** x

init_condition = lambda x: e ** x
left_condition = lambda t: log(t + 1) + 1
right_condition = lambda t: 3 + log(t + 1)

line_limit = log(3)
time_limit = 1

true_solution = lambda x, t: e ** x + log(t + 1)

In [19]:
# second_data = pd.DataFrame(product(range(50, 1000 + 1, 50), [1, 1 / 2]), columns=['n', 'sigma'])
# second_data['error'] = second_data.apply(
#     func=lambda row: banchmark(
#         kappa,
#         f,
#         line_limit,
#         time_limit,
#         init_condition,
#         left_condition,
#         right_condition,
#         true_solution,
#         int(row.n),
#         row.sigma,
#     ),
#     axis=1,
# )
# second_data.to_csv('second.csv', index=False)

second_data = pd.read_csv('second.csv')
second_data

,n,sigma,error
0,50,1.0,5.503425e-04
1,50,0.5,1.095700e-05
2,100,1.0,2.796694e-04
3,100,0.5,2.684406e-06
4,150,1.0,1.874436e-04
5,150,0.5,1.185039e-06
6,200,1.0,1.409679e-04
7,200,0.5,6.643783e-07
8,250,1.0,1.129657e-04
9,250,0.5,4.243499e-07


In [20]:
compare_error(second_data)

Unsupported

In [21]:
number_of_dimensional_layers = 50
number_of_time_layers = 50
sigma = 1

true_grid = get_grid_by_solution(true_solution, line_limit, time_limit, number_of_dimensional_layers, number_of_time_layers)

actual_grid = solve_heat_equation(
    kappa=kappa, 
    f=f, 
    line_limit=line_limit, 
    time_limit=time_limit, 
    number_of_dimensional_layers=number_of_dimensional_layers, 
    number_of_time_layers=number_of_time_layers, 
    init_condition=init_condition, 
    left_condition=left_condition, 
    right_condition=right_condition, 
    sigma=sigma,
)

show_error(true_grid, actual_grid, line_limit, time_limit)
show_heat_map(actual_grid)
compare_grids(actual_grid, true_grid, line_limit, time_limit)

Error: 0.0005503425228050585
Bad point: (24, 8) = (0.5380958148578496, 0.16326530612244897)



Unsupported

In [22]:
number_of_dimensional_layers = 10
number_of_time_layers = 270
sigma = 0

true_grid = get_grid_by_solution(true_solution, line_limit, time_limit, number_of_dimensional_layers, number_of_time_layers)

actual_grid = solve_heat_equation(
    kappa=kappa, 
    f=f, 
    line_limit=line_limit, 
    time_limit=time_limit, 
    number_of_dimensional_layers=number_of_dimensional_layers, 
    number_of_time_layers=number_of_time_layers, 
    init_condition=init_condition, 
    left_condition=left_condition, 
    right_condition=right_condition, 
    sigma=sigma,
)

show_error(true_grid, actual_grid, line_limit, time_limit)
check_steps(line_limit, time_limit, number_of_dimensional_layers, number_of_time_layers, kappa)
show_heat_map(actual_grid)
compare_grids(actual_grid, true_grid, line_limit, time_limit)

Error: 0.0004267628598313422
Bad point: (5, 62) = (0.6103401603711721, 0.23048327137546468)

h: 0.12206803207423442
tau: 0.0037174721189591076

2 * kappa * tau: 0.01486988847583643
h^2: 0.014900604454476322

Условие устойчивости выполняется!



Unsupported

In [23]:
number_of_dimensional_layers = 10
number_of_time_layers = 253 # -17
sigma = 0

true_grid = get_grid_by_solution(true_solution, line_limit, time_limit, number_of_dimensional_layers, number_of_time_layers)

actual_grid = solve_heat_equation(
    kappa=kappa, 
    f=f, 
    line_limit=line_limit, 
    time_limit=time_limit, 
    number_of_dimensional_layers=number_of_dimensional_layers, 
    number_of_time_layers=number_of_time_layers, 
    init_condition=init_condition, 
    left_condition=left_condition, 
    right_condition=right_condition, 
    sigma=sigma,
)

show_error(true_grid, actual_grid, line_limit, time_limit)
check_steps(line_limit, time_limit, number_of_dimensional_layers, number_of_time_layers, kappa)
show_heat_map(actual_grid)
compare_grids(actual_grid, true_grid, line_limit, time_limit)

Error: 1.9354122182350708
Bad point: (5, 252) = (0.6103401603711721, 1.0)

h: 0.12206803207423442
tau: 0.003968253968253968

2 * kappa * tau: 0.015873015873015872
h^2: 0.014900604454476322

Условие устойчивости не выполняется!



Unsupported